#Paquetes necesarios

In [ ]:
import cv2  
import math 
import easyocr
import pytesseract
import time
import csv
import os
import csv
import torch
from ultralytics import YOLO

In [ ]:
torch.cuda.empty_cache()

model = YOLO('yolo11s.pt') 

model.train(
    data='dataset/data.yaml',
    epochs=50,
    imgsz=960,
    batch=4,
    name='matriculas_yolo_v2',
    device='cuda',
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5, 
    translate=0.10,
    scale=0.6,  
    shear=2.0,
    flipud=0.3,
    mixup=0.1,
    mosaic=1.0, 
)


torch.cuda.empty_cache()

model = YOLO('runs/detect/matriculas_yolo_v2/weights/best.pt')

model.train(
    data='dataset/data.yaml',
    epochs=20,
    imgsz=960,
    batch=2,  
    device='cuda',
    mosaic=0.0,  
    mixup=0.0,
    degrees=0,
    translate=0.05,
    scale=0.4,
    shear=0,
    flipud=0.1,
    name='matriculas_yolo_v2_finetuned'
)

In [3]:

# ----------------- CARGA DE LOS MODELOS -----------------
model_objects = YOLO('yolo11s.pt')  # personas/vehículos (COCO)
model_plates  = YOLO('runs/detect/matriculas_yolo_v2_finetuned/weights/best.pt')

# ------------------ CARGA DE LOS OCR -------------------
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
easy = easyocr.Reader(['es','en'], gpu=True)

# ----------------- PREPARAR EL VIDEO -----------------
cap = cv2.VideoCapture("video_matriculas.mp4")
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(
    "resultado_final.mp4", fourcc, 30,
    (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
)

# ----------------- PREPARAR CSV -----------------
csv_file = open("detecciones_tracking.csv", "w", newline="", encoding="utf-8")
writer = csv.writer(csv_file)
writer.writerow([
    "frame","tipo_objeto","conf","track_id","x1","y1","x2","y2",
    "conf_matricula","mx1","my1","mx2","my2","texto_matricula"
])

# ----------------- CONTEO POR ID -----------------
CLASSES_CONTAR = {"person","car","truck","motorcycle","bus"}
seen_ids = {c:set() for c in CLASSES_CONTAR}  
counts    = {c:0 for c in CLASSES_CONTAR} 

COLORS = {
    "person": (255, 0, 0),
    "car": (0, 255, 0),
    "truck": (255, 0, 255),
    "bus": (0, 255, 255),
    "motorcycle": (0, 128, 255)
}

next_id = {c: 0 for c in CLASSES_CONTAR}
class_track_ids = {}  


# ----------------- PROCESAR EL VIDEO FRAME A FRAME -----------------
frame_n = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model_objects.track(
        frame, conf=0.35, iou=0.45, tracker="bytetrack.yaml", persist=True
    )

    if results[0].boxes.id is None:
        y = 30
        for k in ["person","car","truck","motorcycle","bus"]:
            cv2.putText(frame, f"{k}: {counts[k]}", (10, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            y += 28
        out.write(frame)
        frame_n += 1
        continue

    boxes  = results[0].boxes.xyxy.cpu().numpy()
    ids    = results[0].boxes.id.cpu().numpy()
    confs  = results[0].boxes.conf.cpu().numpy()
    classes= results[0].boxes.cls.cpu().numpy()

    for box, track_id, conf, cls in zip(boxes, ids, confs, classes):
        cls_name = model_objects.model.names[int(cls)]
        if cls_name not in CLASSES_CONTAR:
            continue

        x1,y1,x2,y2 = map(int, box)

        # ---------- CONTEO ÚNICO ----------
        if int(track_id) not in seen_ids[cls_name]:
            seen_ids[cls_name].add(int(track_id))
            counts[cls_name] += 1
        
        orig_id = int(track_id)
        key = (cls_name, orig_id)

        if key not in class_track_ids:
            class_track_ids[key] = f"{cls_name[0].upper()}-{next_id[cls_name]}"
            next_id[cls_name] += 1

        disp_id = class_track_ids[key]

        # ---------- DIBUJAR CUADRADOS ----------
        color = COLORS[cls_name]
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame, f"{cls_name} {disp_id} {conf:.2f}", 
                    (x1, max(20, y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        if cls_name == "person":
            face_h = int((y2 - y1) * 0.35)  # parte superior del cuerpo
            face_region = frame[y1:y1+face_h, x1:x2]
            # Pixelar
            if face_region.size > 0:
                face_small = cv2.resize(face_region, (10, 10), interpolation=cv2.INTER_LINEAR)
                face_pixel = cv2.resize(face_small, (x2-x1, face_h), interpolation=cv2.INTER_NEAREST)
                frame[y1:y1+face_h, x1:x2] = face_pixel

        # ---------- DETECTAR MATRÍCULAS ----------
        texto_matricula = ""
        mx1=my1=mx2=my2=conf_m=0.0

        if cls_name in {"car","truck","bus","motorcycle"}:
            crop_vehicle = frame[y1:y2, x1:x2]
            plate_results = model_plates(crop_vehicle, conf=0.22)

            if len(plate_results) and plate_results[0].boxes is not None:
                for p in plate_results[0].boxes:
                    px1,py1,px2,py2 = map(int, p.xyxy[0])
                    mx1, my1, mx2, my2 = x1+px1, y1+py1, x1+px2, y1+py2
                    conf_m = float(p.conf[0])

                    crop_plate = frame[my1:my2, mx1:mx2]
                    crop_plate = cv2.resize(crop_plate, None, fx=3.5, fy=3.5, interpolation=cv2.INTER_CUBIC)
                    crop_plate = cv2.cvtColor(crop_plate, cv2.COLOR_BGR2GRAY)

                    easy_res = easy.readtext(crop_plate, detail=0)
                    texto_matricula = easy_res[0] if len(easy_res)>0 else ""

                    cv2.rectangle(frame, (mx1,my1),(mx2,my2),(0,255,255),2)
                    cv2.putText(frame, texto_matricula, (mx1, max(20,my1-6)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0,255,255),2)

        # ---------- ESCRIBIR CSV ----------
        writer.writerow([
            frame_n, cls_name, round(float(conf),3), int(track_id),
            x1,y1,x2,y2,
            round(float(conf_m),3), mx1,my1,mx2,my2, texto_matricula
        ])

    # ---------- MOSTRAR CONTEO ----------
    y = 30
    for k in ["person","car","truck","motorcycle","bus"]:
        cv2.putText(frame, f"{k}: {counts[k]}", (10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        y += 28

    out.write(frame)
    frame_n += 1

cap.release()
out.release()
csv_file.close()

print("Vídeo exportado: resultado_final.mp4")
print("CSV exportado: detecciones_tracking.csv")
print("Conteo único final:", {k: len(v) for k,v in seen_ids.items()})
print("Totales (mismo valor):", counts)



0: 384x640 3 cars, 1 bus, 1 bench, 218.4ms
Speed: 10.2ms preprocess, 218.4ms inference, 27.1ms postprocess per image at shape (1, 3, 384, 640)

0: 800x960 (no detections), 142.4ms
Speed: 7.6ms preprocess, 142.4ms inference, 1.3ms postprocess per image at shape (1, 3, 800, 960)

0: 704x960 (no detections), 159.0ms
Speed: 5.9ms preprocess, 159.0ms inference, 1.2ms postprocess per image at shape (1, 3, 704, 960)

0: 608x960 (no detections), 168.0ms
Speed: 5.6ms preprocess, 168.0ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 960)

0: 672x960 (no detections), 142.7ms
Speed: 7.0ms preprocess, 142.7ms inference, 1.0ms postprocess per image at shape (1, 3, 672, 960)

0: 384x640 3 cars, 1 bus, 1 bench, 22.6ms
Speed: 3.8ms preprocess, 22.6ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 800x960 (no detections), 28.3ms
Speed: 7.0ms preprocess, 28.3ms inference, 0.9ms postprocess per image at shape (1, 3, 800, 960)

0: 704x960 (no detections), 26.4ms
Speed: 

In [ ]:


# Ruta del ejecutable tesseract (ajústala si es diferente)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# OCRs
easy = easyocr.Reader(['en', 'es'])

# Carpeta con las imágenes
input_folder = "Comparativa/"

# CSV donde pondrás las matrículas reales
ground_truth_file = "matriculas_reales.csv"

# CSV donde guardaremos los resultados
output_file = "comparativa_ocr_imgs.csv"

# Leer matrícula real por imagen
gt = {}
with open(ground_truth_file, "r") as f:
    reader = csv.reader(f)
    next(reader)
    for fila in reader:
        imagen, matricula = fila
        gt[imagen] = matricula.strip().upper()

# Crear CSV de resultados
with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["imagen", "matricula_real", "easyocr", "tesseract",
                     "acierto_easyocr", "acierto_tesseract",
                     "tiempo_easyocr(ms)", "tiempo_tesseract(ms)"])

    for img_name in os.listdir(input_folder):

        path = os.path.join(input_folder, img_name)
        img = cv2.imread(path)

        # EASYOCR
        t0 = time.time()
        easy_res = easy.readtext(img)
        easy_text = easy_res[0][-2].replace(" ", "") if easy_res else ""
        easy_time = (time.time() - t0) * 1000

        # TESSERACT
        t1 = time.time()
        tess_text = pytesseract.image_to_string(img).strip().replace("\n", "").replace(" ", "")
        tess_time = (time.time() - t1) * 1000

        # Normalizar
        easy_text = easy_text.upper()
        tess_text = tess_text.upper()
        real = gt[img_name]

        # Comparaciones exactas
        acierto_easy = 1 if real == easy_text else 0
        acierto_tess = 1 if real == tess_text else 0

        # Guardar fila
        writer.writerow([img_name, real, easy_text, tess_text,
                         acierto_easy, acierto_tess,
                         round(easy_time), round(tess_time)])

print("Comparativa terminada. Archivo guardado en:", output_file)


Comparativa terminada. Archivo guardado en: comparativa_ocr_imgs.csv
